In [1]:
from datasets import load_dataset
from transformers import pipeline
import pandas as pd
import torch

torch.set_grad_enabled(False)

[2023-08-26 23:10:03,218] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
device = "cuda:0"

In [3]:
emotion_model = pipeline('text-classification', 'bhadresh-savani/distilbert-base-uncased-emotion', device=device)
detox_model = pipeline('text2text-generation', 's-nlp/bart-base-detox', device=device)

In [4]:
df = pd.read_csv("emotion_toxic.csv")
df

,Unnamed: 0,text,label,label_text,toxicity
0,0,i didnt feel humiliated,0,sadness,0.000217
1,1,i can go from feeling so hopeless to so damned...,0,sadness,0.002481
2,2,im grabbing a minute to post i feel greedy wrong,3,anger,0.005100
3,3,i am ever feeling nostalgic about the fireplac...,2,love,0.000143
4,4,i am feeling grouchy,3,anger,0.000436
...,...,...,...,...,...
15995,15995,i just had a very brief time in the beanbag an...,0,sadness,0.011353
15996,15996,i am now turning and i feel pathetic that i am...,0,sadness,0.020782
15997,15997,i feel strong and good overall,1,joy,0.000151
15998,15998,i feel like this was such a rude comment and i...,3,anger,0.001461


In [5]:
toxic_df = df[df.toxicity > 0.1]
print(toxic_df.shape)


def detoxify(x):
    de = detox_model(x, max_new_tokens=64, num_beams=4)[0]['generated_text']
    # print(x, de)
    return de

# detoxify(toxic_df.text.iloc[0])
toxic_df["detoxified"] = toxic_df.text.map(detoxify)

(782, 5)


Both `max_new_tokens` (=64) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma

In [6]:

def classify_emotion(x):
    x = emotion_model(x)
    return x[0]['label']

toxic_df["pred_emotion"] = toxic_df["text"].map(classify_emotion)
toxic_df["detoxified_emotion"] = toxic_df["detoxified"].map(classify_emotion)

/home/heegyu/transformers/src/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/tmp/ipykernel_1116585/1350102630.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic_df["pred_emotion"] = toxic_df["text"].map(classify_emotion)
/home/heegyu/transformers/src/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/tmp/ipykernel_1116585/1350102630.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [7]:
(toxic_df["label_text"] == toxic_df["pred_emotion"]).mean()

0.9948849104859335

In [8]:
(toxic_df["pred_emotion"] == toxic_df["detoxified_emotion"]).mean()

0.8056265984654731

In [9]:
(toxic_df["label_text"] == toxic_df["detoxified_emotion"]).mean()

0.8043478260869565

In [12]:
toxic_df["detox_success"] = (toxic_df["label_text"] == toxic_df["detoxified_emotion"])
toxic_df.groupby("label_text").agg({"detox_success": ["mean", "sum"]})

/tmp/ipykernel_1116585/327756581.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic_df["detox_success"] = (toxic_df["label_text"] == toxic_df["detoxified_emotion"])


detox_success     
                    mean  sum
label_text                   
anger           0.581673  146
fear            0.848485   56
joy             0.963636  106
love            0.909091   50
sadness         0.897887  255
surprise        1.000000   16